In [ ]:
class Solution {
    int memo[9][9][9][9][3];
    pair<int,int>mouse;
    pair<int,int>cat;
    pair<int,int>food;    
    int catJump;
    int mouseJump;
public:
    bool canMouseWin(vector<string>& grid, int catJump, int mouseJump) 
    {
        int m = grid.size();
        int n = grid[0].size();
        queue<array<int,5>>q;
        this->catJump = catJump;
        this->mouseJump = mouseJump;

        for (int i=0; i<m; i++)
            for (int j=0; j<n; j++)
            {
                if (grid[i][j]=='M')
                    mouse = {i,j};
                if (grid[i][j]=='C')
                    cat = {i,j};
                if (grid[i][j]=='F')
                    food = {i,j};
            }
        
        for (int i=0; i<m; i++)
            for (int j=0; j<n; j++)
            {
                memo[i][j][i][j][1] = 2; // cat win
                memo[i][j][i][j][2] = 2; // cat win                                
                q.push({i, j, i, j, 1}); 
                q.push({i, j, i, j, 2}); 
                if (i==food.first && j==food.second) continue;
                memo[food.first][food.second][i][j][1] = 1; // mouse win
                memo[food.first][food.second][i][j][2] = 1; // mouse win
                memo[i][j][food.first][food.second][1] = 2; // cat win                
                memo[i][j][food.first][food.second][2] = 2; // cat win                
                q.push({food.first, food.second, i, j, 1}); 
                q.push({food.first, food.second, i, j, 2}); 
                q.push({i, j, food.first, food.second, 1});                   
                q.push({i, j, food.first, food.second, 2});  
            }
        
       
        
        int step = 0;
        while (!q.empty())
        {            
            step++;
            if (step>160) return false;
            int len = q.size();
            while (len--)
            {
                int mx = q.front()[0];
                int my = q.front()[1];
                int cx = q.front()[2];
                int cy = q.front()[3];
                int t = q.front()[4];   
                int status = memo[mx][my][cx][cy][t];
                q.pop();
                
                for (auto nextNode: findAllAdjacents(grid,mx,my,cx,cy,t))
                {
                    int mx2 = nextNode[0];
                    int my2 = nextNode[1];
                    int cx2 = nextNode[2];
                    int cy2 = nextNode[3];                                        
                    int t2 = nextNode[4];

                    if (memo[mx2][my2][cx2][cy2][t2]!=0) continue; // has been determined

                    if (t2==status)   // immediate win, (m2,c2,t2)->(m,c,t)
                    {
                        memo[mx2][my2][cx2][cy2][t2] = status;
                        q.push({mx2,my2,cx2,cy2,t2});
                    }
                    else if (allAdjacentsWin(grid, mx2,my2,cx2,cy2,t2))    // eventually lose
                    {
                        memo[mx2][my2][cx2][cy2][t2] = (t2==1)? 2:1;
                        q.push({mx2,my2,cx2,cy2,t2});              
                    }                
                }                
            }
        }
        
        return memo[mouse.first][mouse.second][cat.first][cat.second][1]==1;        
    }
    
    vector<vector<int>> findAllAdjacents(vector<string>& grid, int mx, int my, int cx, int cy, int t)
    {
        int m = grid.size();
        int n = grid[0].size();
        auto dir = vector<pair<int,int>>({{1,0},{-1,0},{0,1},{0,-1}});
        vector<vector<int>>neighbours;
        if (t==1)
        {
            for (int k=0; k<4; k++)
                for (int a = 0; a<=catJump; a++)
                {
                    int cx2 = cx+dir[k].first*a;
                    int cy2 = cy+dir[k].second*a;
                    if (cx2<0||cx2>=m||cy2<0||cy2>=n) break;
                    if (grid[cx2][cy2]=='#') break;
                    neighbours.push_back({mx,my,cx2,cy2,2});
                }
        }
        else
        {
            for (int k=0; k<4; k++)
                for (int a = 0; a<=mouseJump; a++)
                {
                    int mx2 = mx+dir[k].first*a;
                    int my2 = my+dir[k].second*a;
                    if (mx2<0||mx2>=m||my2<0||my2>=n) break;
                    if (grid[mx2][my2]=='#') break;
                    neighbours.push_back({mx2,my2,cx,cy,1});
                }
        }
        return neighbours;
    }
    
    bool allAdjacentsWin(vector<string>& grid, int mx, int my, int cx, int cy, int t)
    {
        int m = grid.size();
        int n = grid[0].size();
        auto dir = vector<pair<int,int>>({{1,0},{-1,0},{0,1},{0,-1}});
        
        if (t==1)
        {
            for (int k=0; k<4; k++)
                for (int a = 0; a<=mouseJump; a++)
                {
                    int mx2 = mx+dir[k].first*a;
                    int my2 = my+dir[k].second*a;
                    if (mx2<0||mx2>=m||my2<0||my2>=n) break;
                    if (grid[mx2][my2]=='#') break;
                    if (memo[mx2][my2][cx][cy][2]!=2)
                        return false;
                }                
        }
        else if (t==2)
        {
            for (int k=0; k<4; k++)
                for (int a = 0; a<=catJump; a++)
                {
                    int cx2 = cx+dir[k].first*a;
                    int cy2 = cy+dir[k].second*a;
                    if (cx2<0||cx2>=m||cy2<0||cy2>=n) break;
                    if (grid[cx2][cy2]=='#') break;
                    if (memo[mx][my][cx2][cy2][1]!=1)
                        return false;
                }
        }
        return true;
    }    
};



In [172]:
from typing import List
from collections import deque

class Solution:
    def canMouseWin(self, grid: List[str], catJump: int, mouseJump: int) -> bool:
        def find_previous(mx, my, cx, cy, turn):
            # 寻找在当前turn下，哪些状态可以到达（mx, my, cx, cy）
            # 在状态（mx, my, cx, cy），是谁先走的，turn==1是老鼠，=2的时候是猫
            # 那么前一个状态就是相反的
            postions = []
            dirs = [(1, 0), (0, 1), (0, -1), (-1, 0)]
            if turn == 1: # 作为前一轮，如果turn==1，那么前一轮与之相反
                for tx, ty in dirs:  # 猫走
                    for step in range(catJump + 1):
                        cx2, cy2 = cx + tx * step, cy + ty * step
                        if cx2 < 0 or cx2 >= rows or cy2 < 0 or cy2 >= cols:
                            break
                        if grid[cx2][cy2] == '#':
                            break
                        postions.append((mx, my, cx2, cy2, 2))
            else: # 老鼠走
                for tx, ty in dirs:  
                    for step in range(mouseJump + 1):
                        mx2, my2 = mx + tx * step, my + ty * step
                        if mx2 < 0 or mx2 >= rows or my2 < 0 or my2 >= cols:
                            break
                        if grid[mx2][my2] == '#':
                            break
                        postions.append((mx2, my2, cx, cy, 1))
            return postions
        
        def all_next_win(mx, my, cx, cy, turn):
            # 寻找当前状态的下一个状态
            # 由于是倒退的，所以我们向下走的每一个状态，都在 memo中有记载
            dirs = [(1, 0), (0, 1), (0, -1), (-1, 0)]
            if turn == 1: # 因为是寻找下一个状态，所以是以turn的状态找
                 for tx, ty in dirs:  # 老鼠走
                    for step in range(mouseJump + 1):
                        mx2, my2 = mx + tx * step, my + ty * step
                        if mx2 < 0 or mx2 >= rows or my2 < 0 or my2 >= cols:
                            break
                        if grid[mx2][my2] == '#':
                            break
                        # 以老鼠的步伐走，如果走出一个猫不是必赢的状态，那么就是False, 也就是老鼠不是必输
                        state = (mx2, my2, cx, cy, 2)
                        if state not in memo or memo[state] != 2:
                            return False
            else:
                for tx, ty in dirs:  # 猫走
                    for step in range(catJump + 1):
                        cx2, cy2 = cx + tx * step, cy + ty * step
                        if cx2 < 0 or cx2 >= rows or cy2 < 0 or cy2 >= cols:
                            break
                        if grid[cx2][cy2] == '#':
                            break
                        state = (mx, my, cx2, cy2, 1)
                        if state not in memo or memo[state] != 1:
                            return False
            return True
        
        rows, cols = len(grid), len(grid[0])
        cp, mp, fp = None, None, None # 猫、鼠、食物的所在位置
        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == 'C':
                    cp = (i, j)
                elif grid[i][j] == 'M':
                    mp = (i, j)
                elif grid[i][j] == 'F':
                    fp = (i, j)
                    
        memo = {}
        dq = deque() # [老鼠的位置，猫的位置，谁先走] ，
        for i in range(rows):
            for j in range(cols):
                for t in range(1, 2+1): # 表示当前状态下，是老鼠先走还是猫先走
                    if grid[i][j] == '#':
                        continue
                    memo[(i, j, i, j, t)] = 2 # 猫和老鼠处于同一个位置，猫赢
                    dq.append((i, j, i, j, t))
                    if (i, j) == fp:
                        continue
                    # 记录了猫和老鼠必赢的状态有哪些
                    memo[(fp[0], fp[1], i, j, t)] = 1 #老鼠赢
                    dq.append((fp[0], fp[1], i, j, t))
                    memo[(i, j, fp[0], fp[1], t)] = 2 #猫赢
                    dq.append((i, j, fp[0], fp[1], t))
        
        step = 0
        while dq:
            step += 1
            if step > 2000: # 大于2000步，猫赢
                return False
            
            for _ in range(len(dq)):
                mx, my, cx, cy, t = dq.popleft() # 当前状态下弹出的老鼠、猫和食物的位置，已经直到谁赢谁输了
                status = memo[(mx, my, cx, cy, t)] # 1 老鼠赢，2猫赢
                pre_pos = find_previous(mx, my, cx, cy, t) # 寻找当前状态的前一个状态
               
                for pos in pre_pos: # 往前推出哪些状态可以走到 (mx, my, cx, cy, t)
                    if pos in memo:
                        continue
                    mx2, my2, cx2, cy2, t2 = pos # 前一轮是老鼠先走，那么老鼠必定走的是可以导致最终赢的state
                    if status == t2: # 必赢状态
                        memo[pos] = status
                        dq.append(pos)
                    elif all_next_win(*pos): # 必输状态
                        # 如果不能判断必赢，那么看看是不是必输
                        # 猫往下走的每个状态都是老鼠赢，那么猫肯定输的
                        # 必输的条件是：当前的pos往前推的任何状态都是赢的
                        memo[pos] = 1 if t2 == 2 else 2
                        dq.append(pos)
                        
        ini_state = (mp[0], mp[1], cp[0], cp[1], 1) # 初始状态
        if ini_state in memo:
            return memo[ini_state] == 1
        return False

In [182]:
from typing import List
from collections import deque
from functools import lru_cache

class Solution:
    def canMouseWin(self, grid: List[str], catJump: int, mouseJump: int) -> bool:
        @lru_cache(None)
        def dp(cat, mouse, step):
            if cat == fp: # 猫先到食物
                return False
            if step == rows*cols*2: # 死循环，猫赢
                return False
            if cat == mouse: # 相同位置，猫赢
                return False
            if mouse == fp:
                return True
            dirs = [(0,1), (1,0), (-1,0), (0,-1)]
            if step % 2 == 0:
                # 老鼠先走
                for tx, ty in dirs:
                    for s in range(mouseJump+1):
                        n_mx, n_my = tx*s + mouse[0], ty*s + mouse[1]
                        if n_mx < 0 or n_mx >= rows or n_my < 0 or n_my >= cols:
                            break
                        if grid[n_mx][n_my] == '#':
                            break
                        # 对于老鼠来说，能走出一条赢的，就算赢
                        if dp(cat, (n_mx, n_my), step + 1):
                            return True
                return False
            else:
                # 猫走
                for tx, ty in dirs:
                    for s in range(catJump+1):
                        n_cx, n_cy = tx*s + cat[0], ty*s + cat[1]
                        if n_cx < 0 or n_cx >= rows or n_cy < 0 or n_cy >= cols:
                            break
                        if grid[n_cx][n_cy] == '#':
                            break
                        # 对于猫来说，如果猫能赢一次，那么老鼠就是必输的
                        if not dp((n_cx, n_cy), mouse, step+1):
                            return False
                # 如果猫都是输的，那么老鼠赢
                return True
        
        
        rows, cols = len(grid), len(grid[0])
        cp, mp, fp = None, None, None # 猫、鼠、食物的所在位置
        for i in range(rows):
            for j in range(cols):
                if grid[i][j] == 'C':
                    cp = (i, j)
                elif grid[i][j] == 'M':
                    mp = (i, j)
                elif grid[i][j] == 'F':
                    fp = (i, j)
        return dp(cp, mp, 0)

In [183]:
solution = Solution()
solution.canMouseWin(grid = ["####F",
                             "#C...",
                             "M...."], catJump = 1, mouseJump = 2)

True